<a href="https://colab.research.google.com/github/dinggga/super-broccoli/blob/main/220516hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[개념정리]**


## **09. 추천 시스템**

### **9.1 추천 시스템의 개요와 배경**
- 연관 콘텐츠 추천 > 해당 사이트 신뢰해 더 많은 컨텐츠 선택 > 데이터 축적되며 정확도 상승하는 선순환 시스템
- 온라인 스토어의 필수 요소. 고객/상품 관련 데이터 이용.
- 추천 시스템 유형
  - 콘텐츠 기반 필터링(Content based filtering)
  - 협업 필터링(Collaborative Filtering): 최근접 이웃(Nearest Neighbor) 협업 필터링, 잠재 요인(Latent Factor) 협업 필터링



### **9.2 콘텐츠 기반 필터링 추천 시스템**
: 사용자가 특정 아이템을 매우 선호하는 경우 그와 비슷한 콘텐츠를 가진 다른 아이템을 추천하는 방식.

### **9.3 최근접 이웃 협업 필터링**
- 협업 필터링
  - 사용자 행동 양식(User Behavior)만을 기반으로 추천 수행
  - 주요 목표: 사용자-아이템 평점 매트릭스와 같은 축적된 사용자 행동 데이터 기반으로 예측평가(Predicted Rating)
- 최근접 이웃 방식 / 잠재 요인 방식으로 나뉨
  - 사용자-아이템 평점 행렬 데이터에만 의지해 추천 수행
  - 열: 개별 아이템 / 행: 개별 사용자
  - 판다스 pivot_table() 등의 함수 통해 행렬 형태로 변경.
  - 희소 행렬(Sparse Matrix) 특성
- 최근접 협업 필터링
  - 메모리(Memory) 협업 필터링이라고도 함. 사용자 기반/아이템 기반으로 다시 나뉨.
  - 사용자 기반(User-User): 특정 사용자와 타 사용자 간의 유사도 측정 후 가장 유사도 높은 TOP-N으로 선정해 추천.
  - 아이템 기반(Item-Item): 사용자들이 아이템을 좋아하는지/싫어하는지의 평가 척도가 유사한 아이템이 추천됨. 대부분 아이템 기반 알고리즘 적용됨.
- 희소행렬이므로 코사인 유사도 주로 이용

### **9.4 잠재 요인 협업 필터링**
- 잠재 요인 협업 필터링의 이해
  - 행렬 분해(Matrix Factorization): 대규모 다차원 행렬을 SVD와 같은 차원 감소 기법으로 분해하는 과정에서 잠재 요인 추출
  - **사용자-아이템 행렬**을 **사용자-잠재요인 행렬** / **잠재요인-아이템 행렬**로 분해 > 두 행렬 내적 통해 새로운 **예측 사용자-아이템 평점 행렬** 만듦
- 행렬 분해의 이해
  - 다차원의 매트릭스를 저차원 매트릭스로 분해하는 기법. SVD(Singular Vector Decompositon), NMF(Non-Negative Matrix Factorization) 등
  - 평점행렬 R(MXN 차원) -행렬분해 통해 -> 사용자-K차원 잠재요인행렬 P(MXK 차원) / K차원 잠재요인-아이템 행렬 Q.T(KXN 차원)
  - R = P * Q.T
  - r(u,i) = p(u)*q.t(i)
  - 행렬 분해: 주로 SVD 방식. but null값 없는 행렬에만 적용 가능. 
    - null값 있는 경우에는 확률적 경사하강법(SGD)이나 ALS(Alternating Least Squares) 방식
- 확률적 경사 하강법을 이용한 행렬 분해
  - P와 Q 행려로 계산된 예측 R 행렬 값이 실제 R 행렬 값과 가장 최소의 오류를 가질 수 있도록 반복적인 비용 함수 최적화를 통해 P와 Q 유추
  - 예측 R 행렬과 실제 R 행렬의 오류값을 계산하며 이를 최소화할 수 있도록 P와 Q 행렬을 적절한 값으로 각각 업데이트. 2, 3번 작업 반복.

---
---
# **[필사]**

In [1]:
# SGD 이용해 행렬 분해 수행
import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P와 Q 초기화, 잠재요인 차원 K는 3 설정. 
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN ],
              [np.NaN, 5, np.NaN, 3, 1 ],
              [np.NaN, np.NaN, 3, 4, 4 ],
              [5, 2, 1, 2, np.NaN ]])
num_users, num_items = R.shape
K=3

# P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 random한 값으로 입력합니다. 
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [2]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
      
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse


In [3]:
# R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장. 
non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]

steps=1000
learning_rate=0.01
r_lambda=0.01

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트. 
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측 값의 차이인 오류 값 구함
        eij = r - np.dot(P[i, :], Q[j, :].T)
        # Regularization을 반영한 SGD 업데이트 공식 적용
        P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
        Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0 :
        print("### iteration step : ", step," rmse : ", rmse)

### iteration step :  0  rmse :  3.2388050277987723
### iteration step :  50  rmse :  0.4876723101369648
### iteration step :  100  rmse :  0.1564340384819247
### iteration step :  150  rmse :  0.07455141311978046
### iteration step :  200  rmse :  0.04325226798579314
### iteration step :  250  rmse :  0.029248328780878973
### iteration step :  300  rmse :  0.022621116143829466
### iteration step :  350  rmse :  0.019493636196525135
### iteration step :  400  rmse :  0.018022719092132704
### iteration step :  450  rmse :  0.01731968595344266
### iteration step :  500  rmse :  0.016973657887570753
### iteration step :  550  rmse :  0.016796804595895633
### iteration step :  600  rmse :  0.01670132290188466
### iteration step :  650  rmse :  0.01664473691247669
### iteration step :  700  rmse :  0.016605910068210026
### iteration step :  750  rmse :  0.016574200475705
### iteration step :  800  rmse :  0.01654431582921597
### iteration step :  850  rmse :  0.01651375177473524
### iterati

In [4]:
pred_matrix = np.dot(P, Q.T)
print('예측 행렬:\n', np.round(pred_matrix, 3))

예측 행렬:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]
